In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import cv2
import numpy as np

# Open the image files.
img1_color = cv2.imread("/content/drive/MyDrive/Img_regis/Example2_moving.tif") # Image to be aligned.
img2_color = cv2.imread("/content/drive/MyDrive/Img_regis/Example2_fixed.tif") # Reference image.

# Convert to grayscale.
img1 = cv2.cvtColor(img1_color, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2_color, cv2.COLOR_BGR2GRAY)
height, width = img2.shape

In [31]:
# Create ORB detector with 5000 features.
orb_detector = cv2.ORB_create(5000)

In [32]:
# Find keypoints and descriptors.
# The first arg is the image, second arg is the mask
# (which is not required in this case).
kp1, d1 = orb_detector.detectAndCompute(img1, None)
kp2, d2 = orb_detector.detectAndCompute(img2, None)

In [33]:
# Match features between the two images.
# We create a Brute Force matcher with
# Hamming distance as measurement mode.
matcher = cv2.BFMatcher(cv2.NORM_HAMMING2, crossCheck = True)

In [34]:
# Match the two sets of descriptors.
matches = matcher.match(d1, d2)
matches = list(matches)

In [35]:
# Sort matches on the basis of their Hamming distance.
matches.sort(key = lambda x: x.distance)

In [36]:
# Take the top 90 % matches forward.
matches = matches[:int(len(matches)*0.75)]
no_of_matches = len(matches)

In [37]:
# Define empty matrices of shape no_of_matches * 2.
p1 = np.zeros((no_of_matches, 2))
p2 = np.zeros((no_of_matches, 2))

In [38]:
for i in range(len(matches)):
  p1[i, :] = kp1[matches[i].queryIdx].pt
  p2[i, :] = kp2[matches[i].trainIdx].pt

In [39]:
# Find the homography matrix.
homography, mask = cv2.findHomography(p1, p2, cv2.RANSAC)

In [40]:
# Use this matrix to transform the
# colored image wrt the reference image.
transformed_img = cv2.warpPerspective(img1_color,
					homography, (width, height))

# Save the output.
cv2.imwrite('output.tif', transformed_img)

True

In [41]:
# Evaluation Metrics
# Compute reprojection error
points1_reproj = cv2.perspectiveTransform(p1.reshape(-1, 1, 2), homography).reshape(-1, 2)
reproj_error = np.mean(np.sqrt(np.sum((p2 - points1_reproj)**2, axis=1)))

In [42]:
print("Reprojection Error:", reproj_error)

Reprojection Error: 3371.088583054692


In [43]:
# Compute feature matching accuracy
matching_accuracy = len(matches) / len(d1)
print("Matching Accuracy:", matching_accuracy)

Matching Accuracy: 0.0812699680511182
